Comenzaremos por importar los módulos necesarios e ingestar los datos

In [15]:
import pandas as pd #Importamos las librerías necesarias
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [16]:
train = pd.read_excel('E-Commerce_train.xlsx') #Ingestamos los .csv
test = pd.read_excel('E-Commerce_test.xlsx')

Intentaremos conocer un poco nuestros datos, y buscaremos valores faltantes o marcadamente fuera de lo común

In [17]:
print('Train:\n',train.isnull().sum())
print(train.describe())
print(train.columns)
for col in train.columns: print(np.sort(train[col].unique()))
print('Test:\n',test.isnull().sum())
print(test.describe())
print(test.columns)
for col in test.columns: print(np.sort(test[col].unique()))

Train:
 ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
Reached.on.Time_Y.N    0
dtype: int64
               ID  Customer_care_calls  Customer_rating  Cost_of_the_Product  \
count  8999.00000          8999.000000      8999.000000          8999.000000   
mean   4500.00000             4.015113         2.994999           208.561507   
std    2597.93187             1.138934         1.413143            48.289172   
min       1.00000             2.000000         1.000000            96.000000   
25%    2250.50000             3.000000         2.000000           168.000000   
50%    4500.00000             4.000000         3.000000           211.000000   
75%    6749.50000             5.000000         4.000000           249.500000   
max    8999.00000             7.000000 

Del análisis exploratorio hemos obtenido que no hay valores faltantes o marcadamente fuera de lo común, pero habiendo percibido una importante diferencia entre los valores de 'Discount_offered' de las tablas, decidimos analizarla

In [18]:
train.groupby('Discount_offered').describe()

ID                                                    \
                  count         mean          std    min      25%     50%   
Discount_offered                                                            
1                 639.0  5752.525822  2095.059751   13.0  4113.00  5903.0   
2                 652.0  5668.384969  2105.410628   23.0  4144.25  5752.5   
3                 643.0  5681.601866  2026.207907    7.0  4316.00  5824.0   
4                 632.0  5711.110759  1985.275069   34.0  4244.25  5738.0   
5                 622.0  5689.824759  2046.596238   99.0  4142.75  5848.0   
...                 ...          ...          ...    ...      ...     ...   
61                 53.0  1561.320755   945.668135   32.0   638.00  1696.0   
62                 49.0  1505.102041   894.171713   37.0   768.00  1582.0   
63                 53.0  1609.584906   881.668249  144.0   978.00  1654.0   
64                 43.0  1826.372093   915.972266   61.0  1104.00  1986.0   
65                 36.0  1448.694444   878.941338  112.0   778.50  1316.0   

                                  Customer_care_calls            ...  \
                      75%     max               count      mean  ...   
Discount_offered                                                 ...   
1                 7513.00  8995.0               639.0  4.078247  ...   
2                 7484.00  8975.0               652.0  4.157975  ...   
3                 7216.50  8996.0               643.0  4.110420  ...   
4                 7409.75  8999.0               632.0  4.072785  ...   
5                 7363.25  8998.0               622.0  4.120579  ...   
...                   ...     ...                 ...       ...  ...   
61                2452.00  3054.0                53.0  3.924528  ...   
62                2217.00  2990.0                49.0  3.734694  ...   
63                2441.00  3006.0                53.0  3.698113  ...   
64                2635.00  3087.0                43.0  3.744186  ...   
65                2171.00  2898.0                36.0  3.944444  ...   

                 Weight_in_gms         Reached.on.Time_Y.N            \
                           75%     max               count      mean   
Discount_offered                                                       
1                      5326.50  5999.0               639.0  0.494523   
2                      5273.75  5986.0               652.0  0.509202   
3                      5252.00  5986.0               643.0  0.486781   
4                      5289.50  6000.0               632.0  0.473101   
5                      5324.00  6102.0               622.0  0.495177   
...                        ...     ...                 ...       ...   
61                     3096.00  3812.0                53.0  1.000000   
62                     3259.00  3997.0                49.0  1.000000   
63                     2751.00  3788.0                53.0  1.000000   
64                     2889.00  3812.0                43.0  1.000000   
65                     3508.25  3984.0                36.0  1.000000   

                                                     
                       std  min  25%  50%  75%  max  
Discount_offered                                     
1                 0.500362  0.0  0.0  0.0  1.0  1.0  
2                 0.500299  0.0  0.0  1.0  1.0  1.0  
3                 0.500214  0.0  0.0  0.0  1.0  1.0  
4                 0.499671  0.0  0.0  0.0  1.0  1.0  
5                 0.500379  0.0  0.0  0.0  1.0  1.0  
...                    ...  ...  ...  ...  ...  ...  
61                0.000000  1.0  1.0  1.0  1.0  1.0  
62                0.000000  1.0  1.0  1.0  1.0  1.0  
63                0.000000  1.0  1.0  1.0  1.0  1.0  
64                0.000000  1.0  1.0  1.0  1.0  1.0  
65                0.000000  1.0  1.0  1.0  1.0  1.0  

[65 rows x 56 columns]

Hemos descubierto que aquellas ventas con descuento mayor al 10% siempre se entregaron a tiempo, pero el dataset de testeo no tiene descuentos mayores al 10%

In [19]:
df = train.loc[train['Discount_offered'] <= 10]
importance = {'low':1,'medium':2,'high':3}
gender = {'F':1,'M':2}
warehouse = {'A':1,'B':2,'C':3,'D':4,'F':5}
shipment = {'Road':1,'Flight':2,'Ship':3}
num = df.replace({'Product_importance':importance,'Gender':gender,'Warehouse_block':warehouse,'Mode_of_Shipment':shipment})
num.describe()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
count,6352.000000,6352.000000,6352.000000,6352.000000,6352.000000,6352.000000,6352.000000,6352.000000,6352.000000,6352.000000,6352.000000,6352.000000
mean,5722.395466,3.327613,2.517947,4.126574,2.995592,213.714421,3.601385,1.602960,1.499370,5.473709,4091.737406,0.483627
std,2034.881336,1.491974,0.756430,1.175930,1.410416,48.471799,1.527201,0.645304,0.500039,2.874688,1570.853596,0.499771
min,4.000000,1.000000,1.000000,2.000000,1.000000,96.000000,2.000000,1.000000,1.000000,1.000000,1001.000000,0.000000
25%,4235.750000,2.000000,2.000000,3.000000,2.000000,173.000000,3.000000,1.000000,1.000000,3.000000,2881.000000,0.000000
50%,5823.500000,3.000000,3.000000,4.000000,3.000000,220.000000,3.000000,2.000000,1.000000,5.000000,4598.000000,0.000000
75%,7411.250000,5.000000,3.000000,5.000000,4.000000,254.000000,4.000000,2.000000,2.000000,8.000000,5294.000000,1.000000
max,8999.000000,5.000000,3.000000,7.000000,5.000000,310.000000,10.000000,3.000000,2.000000,10.000000,6102.000000,1.000000


Hemos normalizado la data de manera numérica para poder modelar y hemos retirado las entradas con descuento mayor al 10%, ahora quitaremos la columna 'Gender' por considerarla irrelevante

In [20]:
num1 = num.drop(columns='Gender')

In [21]:
x = num1.drop(columns='Reached.on.Time_Y.N').values
y = num['Reached.on.Time_Y.N'].values
x.reshape(-1,1)
y.reshape(-1,1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25,random_state=7)

Hemos armado los conjuntos de dato para su modelado. El número 7 en random_state fue seleccionado por ser mi número de la suerte :)

In [22]:
model = DecisionTreeClassifier(max_depth = 7) 
model.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=7)

Creamos y entrenamos el modelo de árbol

In [23]:
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

matrix1 = confusion_matrix(y_train, y_train_pred)
matrix2 = confusion_matrix(y_test,y_test_pred)

print('Matrix1:\n',matrix1,'\nMatrix2:\n',matrix2)

Matrix1:
 [[2342  101]
 [1716  605]] 
Matrix2:
 [[781  56]
 [589 162]]


Hemos utilizado el modelo para predecir, dando como resultado un recall del 92%

In [24]:
importance = {'low':1,'medium':2,'high':3}
gender = {'F':1,'M':2}
warehouse = {'A':1,'B':2,'C':3,'D':4,'F':5}
shipment = {'Road':1,'Flight':2,'Ship':3}
testing = test.replace({'Product_importance':importance,'Gender':gender,'Warehouse_block':warehouse,'Mode_of_Shipment':shipment})
testing1 = testing.drop(columns='Gender')

In [25]:
resultados = model.predict(testing1)
res = pd.DataFrame(resultados,columns=['pred'])
res.to_csv('UMazz.csv',index=False)

/home/umazz/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
